In [1]:
#2 thư viện của thầy hết, chưa biết dùng đâu
import os
import matplotlib.pyplot as plt

In [2]:
#Chưa chỉnh gì cả

def visualize_maze(matrix, bonus, start, end, route=None):
    """
    Args:
      1. matrix: The matrix read from the input file,
      2. bonus: The array of bonus points,
      3. start, end: The starting and ending points,
      4. route: The route from the starting point to the ending one, defined by an array of (x, y), e.g. route = [(1, 2), (1, 3), (1, 4)]
    """
    #1. Define walls and array of direction based on the route
    walls=[(i,j) for i in range(len(matrix)) for j in range(len(matrix[0])) if matrix[i][j]=='x']

    if route:
        direction=[]
        for i in range(1,len(route)):
            if route[i][0]-route[i-1][0]>0:
                direction.append('v') #^
            elif route[i][0]-route[i-1][0]<0:
                direction.append('^') #v        
            elif route[i][1]-route[i-1][1]>0:
                direction.append('>')
            else:
                direction.append('<')

        direction.pop(0)

    #2. Drawing the map
    ax=plt.figure(dpi=100).add_subplot(111)

    for i in ['top','bottom','right','left']:
        ax.spines[i].set_visible(False)

    plt.scatter([i[1] for i in walls],[-i[0] for i in walls],
                marker='X',s=100,color='black')
    
    plt.scatter([i[1] for i in bonus],[-i[0] for i in bonus],
                marker='P',s=100,color='green')

    plt.scatter(start[1],-start[0],marker='*',
                s=100,color='gold')

    if route:
        for i in range(len(route)-2):
            plt.scatter(route[i+1][1],-route[i+1][0],
                        marker=direction[i],color='silver')

    plt.text(end[1],-end[0],'EXIT',color='red',
         horizontalalignment='center',
         verticalalignment='center')
    plt.xticks([])
    plt.yticks([])
    plt.show()

    print(f'Starting point (x, y) = {start[0], start[1]}')
    print(f'Ending point (x, y) = {end[0], end[1]}')
    
    for _, point in enumerate(bonus):
        print(f'Bonus point at position (x, y) = {point[0], point[1]} with point {point[2]}')

In [3]:
#Đọc maze vô mảng 2 chiều
def read_file(file_name):
    f=open(file_name,'r')
    n = int(f.readline())
    bonus_points = []
    for i in range(n):
        x, y, reward = f.readline().strip().split(' ')
        bonus_points.append((x, y, reward))
    matrix = []
    for line in f.read().splitlines():
        matrix.append(list(line))
    f.close()

    return bonus_points, matrix

In [4]:
#Xác định điểm S và E của maze
def start_end_point(matrix):
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == 'S':
                start = (i,j)
            elif i == 0 or j == 0 or i == len(maze) - 1 or j == len(maze[0]) - 1:
                if maze[i][j]==' ':
                    end = (i,j)
    return start, end

In [5]:
#khu de quy
def DFS(matrix, start, end):
    route = []
    
    if start == end:
        route.append(start)
        return route
    
    stack_frontier = []
    list_explore = []
    dfspath = {}
    stack_frontier.append(start)
    while len(stack_frontier) != 0:
        cur_point = stack_frontier.pop()
        if cur_point == end:
            break        
        neighbour = [(cur_point[0]-1,cur_point[1]),(cur_point[0]+1,cur_point[1]),(cur_point[0],cur_point[1]-1),(cur_point[0],cur_point[1]+1)]
        for point_next in neighbour:
            if point_next not in list_explore:
                if matrix[point_next[0]][point_next[1]] == ' ':
                    stack_frontier.append(point_next)
                    list_explore.append(point_next)
                    dfspath[point_next] = cur_point
    cur_point = end
    route.append(end)
    while cur_point != start:
        route.append(dfspath[cur_point])
        cur_point = dfspath[cur_point]
    route.reverse()
    return route

In [15]:
def Greedy(maze, start, end):
    route = []
    if start == end:
        route.append(start)
        return route
    list_close = []
    list_close.append(start)   
    list_open = []
    list_heu_open = []
    heu_matrix= []
    path = {}
    for i in range(len(maze)):
        row_man_dis = []
        for j in range(len(maze[0])):
            row_man_dis.append(abs(i-end[0])+abs(j-end[1]))
        heu_matrix.append(row_man_dis)
    cur_point = start
    while cur_point != end:
        neighbour = [(cur_point[0]-1,cur_point[1]),(cur_point[0]+1,cur_point[1]),(cur_point[0],cur_point[1]-1),(cur_point[0],cur_point[1]+1)]
        for point_next in neighbour:
            if point_next not in list_close and point_next not in list_open:
                if maze[point_next[0]][point_next[1]] == ' ':
                    list_open.insert(0,point_next)
                    list_heu_open.insert(0,heu_matrix[point_next[0]][point_next[1]])
                    path[point_next] = cur_point
        min_heu = min(list_heu_open)
        index_next = list_heu_open.index(min_heu)
        list_heu_open.pop(index_next)
        cur_point = list_open.pop(index_next)
        list_close.append(cur_point)
    cur_point = end
    route.append(end)
    while cur_point != start:
        route.append(path[cur_point])
        cur_point = path[cur_point]
    route.reverse()
    return route       

In [25]:

diem, maze = read_file("non_bonus/maze6.txt")
start, end = start_end_point(maze)
route = []
route = DFS(maze, start, end)
print(route)

[(4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (6, 14), (6, 15), (6, 16), (7, 16), (7, 17), (7, 18), (6, 18), (5, 18), (5, 19)]


In [26]:
diem, maze = read_file("non_bonus/maze6.txt")
start, end = start_end_point(maze)
route = []
route = Greedy(maze, start, end)
print(route)

[(4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (6, 14), (6, 15), (6, 16), (5, 16), (5, 17), (5, 18), (5, 19)]
